In [2]:
print("hello world")

hello world


In [9]:
import json

f = open('rikishi.json')

# returns JSON object as a dictionary
data = json.load(f)
data = data["records"]
data

[{'id': 7522,
  'sumodbId': 10438,
  'nskId': 0,
  'shikonaEn': 'Hakushuzan',
  'heya': '-',
  'birthDate': '0001-01-01T00:00:00Z',
  'shusshin': '-',
  'debut': '195401',
  'intai': '1959-05-01T00:00:00Z',
  'updatedAt': '2024-02-12T04:54:54.762Z'},
 {'id': 2973,
  'sumodbId': 5772,
  'nskId': 0,
  'shikonaEn': 'Wakahokkai Gentaro',
  'heya': 'Hanakago',
  'birthDate': '1943-03-02T00:00:00Z',
  'shusshin': 'Hokkaido, Kamikawa-gun, Higashikagura-cho',
  'debut': '195907',
  'intai': '1966-09-01T00:00:00Z',
  'updatedAt': '2024-02-12T04:54:54.846Z'},
 {'id': 6428,
  'sumodbId': 6679,
  'nskId': 0,
  'shikonaEn': 'Kakuryuasahi',
  'heya': 'Oshima',
  'birthDate': '1971-02-24T00:00:00Z',
  'shusshin': 'Osaka-fu, Osaka-shi, Tsurumi-ku',
  'height': 178.5,
  'weight': 111,
  'debut': '198705',
  'intai': '1988-11-01T00:00:00Z',
  'updatedAt': '2024-02-12T04:54:54.936Z'},
 {'id': 3683,
  'sumodbId': 381,
  'nskId': 0,
  'shikonaEn': 'Katsumayama Shuichi',
  'heya': 'Naruto',
  'birthDate': '

In [12]:
import pandas as pd

df = pd.DataFrame(data)
df

,id,sumodbId,nskId,shikonaEn,heya,birthDate,shusshin,debut,intai,updatedAt,height,weight,shikonaJp,currentRank,createdAt
0,7522,10438.0,0,Hakushuzan,-,0001-01-01T00:00:00Z,-,195401,1959-05-01T00:00:00Z,2024-02-12T04:54:54.762Z,NaN,NaN,NaN,NaN,NaN
1,2973,5772.0,0,Wakahokkai Gentaro,Hanakago,1943-03-02T00:00:00Z,"Hokkaido, Kamikawa-gun, Higashikagura-cho",195907,1966-09-01T00:00:00Z,2024-02-12T04:54:54.846Z,NaN,NaN,NaN,NaN,NaN
2,6428,6679.0,0,Kakuryuasahi,Oshima,1971-02-24T00:00:00Z,"Osaka-fu, Osaka-shi, Tsurumi-ku",198705,1988-11-01T00:00:00Z,2024-02-12T04:54:54.936Z,178.5,111.0,NaN,NaN,NaN
3,3683,381.0,0,Katsumayama Shuichi,Naruto,1978-04-05T00:00:00Z,"Ehime-ken, Uma-gun, Shingu-mura - Ehime-ken, S...",199403,2005-05-01T00:00:00Z,2024-02-12T04:54:55.023Z,177.0,113.5,NaN,NaN,NaN
4,6560,7299.0,0,Otsuru,Takadagawa,1967-05-14T00:00:00Z,"Tokyo-to, Edogawa-ku",198303,1987-09-01T00:00:00Z,2024-02-12T04:54:55.108Z,186.0,77.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8927,8929,NaN,4265,Nakayama,Asakayama,2008-05-16T15:00:00Z,Saitama,202405,NaN,2024-05-01T06:43:12.975Z,175.0,134.0,中山(なかやま),Jonokuchi 17 West,2024-05-01T06:43:12.966Z
8928,8930,NaN,4271,Hara,Hakkaku,2007-04-25T15:00:00Z,Shimane,202405,NaN,2024-05-01T06:43:16.729Z,170.0,110.0,原(はら),Jonokuchi 19 West,2024-05-01T06:43:16.718Z
8929,8931,NaN,4275,Okaryu,Tatsunami,2001-10-08T15:00:00Z,Aomori,202405,NaN,2024-05-12T09:00:30.163Z,175.0,160.0,大花竜(おおかりゅう),Sandanme 90 East,2024-05-12T09:00:30.162Z
8930,8932,NaN,4243,Seihakuho,Isegahama,2000-08-23T15:00:00Z,Mongolia,202405,NaN,2024-05-12T09:00:35.915Z,192.0,127.0,聖白鵬(せいはくほう),Jonokuchi 3 West,2024-05-12T09:00:35.912Z


In [40]:
# GET /api/basho/:bashoId/torikumi/:division/:day
base_url = "https://sumo-api.com/api"

first_year = 1958
last_year = 2024
months = ["01", "03", "05", "07", "09", "11"]
first_day = 1
last_day = 15
divisions = ["makuuchi", "juryo", "makushita", "sandanme", "jonidan", "jonokuchi"]

excluded_basho_ids = {"195801", "202407", "202409", "202411"} # tournaments in rage that don't have data

years = list(range(first_year, last_year + 1))
days = list(range(first_day, last_day + 1))

params = [(str(year) + month, division, str(day)) for year in years for month in months for day in days for division in divisions]
params = [param for param in params if param[0] not in excluded_basho_ids]

['https://sumo-api.com/api/basho/202405/torikumi/makuuchi/15',
 'https://sumo-api.com/api/basho/202405/torikumi/juryo/15',
 'https://sumo-api.com/api/basho/202405/torikumi/makushita/15',
 'https://sumo-api.com/api/basho/202405/torikumi/sandanme/15',
 'https://sumo-api.com/api/basho/202405/torikumi/jonidan/15',
 'https://sumo-api.com/api/basho/202405/torikumi/jonokuchi/15']

In [43]:
import requests

for param in params[-1:]:
    basho_id, division, day = param
    url = f"{base_url}/basho/{basho_id}/torikumi/{division}/{day}"
    response = requests.get(url)
    data = response.text
    print(data)
    

{
  "date": "202405",
  "startDate": "2024-05-12T00:00:00Z",
  "endDate": "2024-05-26T00:00:00Z",
  "yusho": [
    {
      "type": "Makuuchi",
      "rikishiId": 8879,
      "shikonaEn": "Onosato",
      "shikonaJp": "大の里"
    },
    {
      "type": "Juryo",
      "rikishiId": 12,
      "shikonaEn": "Wakatakakage",
      "shikonaJp": "若隆景　渥"
    },
    {
      "type": "Makushita",
      "rikishiId": 82,
      "shikonaEn": "Fujiseiun",
      "shikonaJp": "藤青雲(ふじせいうん)"
    },
    {
      "type": "Sandanme",
      "rikishiId": 8888,
      "shikonaEn": "Kyokukaiyu"
    },
    {
      "type": "Jonidan",
      "rikishiId": 398,
      "shikonaEn": "Asaazuma",
      "shikonaJp": "朝東(あさあずま)"
    },
    {
      "type": "Jonokuchi",
      "rikishiId": 8908,
      "shikonaEn": "Noda",
      "shikonaJp": "野田(のだ)"
    }
  ],
  "specialPrizes": [
    {
      "type": "Shukun-sho",
      "rikishiId": 8879,
      "shikonaEn": "Onosato",
      "shikonaJp": "大の里"
    },
    {
      "type": "Kanto-sho",
  